In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rubensvectomobile_gmai

In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from keras.models import Model

l = tf.keras.layers

model = tf.keras.Sequential([
    l.Flatten(input_shape=(784,)),
    l.Dense(128, activation='relu'),
    l.Dense(128, activation='relu'),
    l.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

model.summary()

W0731 11:42:55.184903 140511422424832 deprecation.py:506] From /home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(x_train.reshape(-1,784),pd.get_dummies(y_train),nb_epoch=15,batch_size=128,verbose=1)

W0730 15:11:20.228803 140455116187392 training.py:593] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/15
60000/60000 [==============================] - 2s 28us/sample - loss: 2.9863 - acc: 0.8697
Epoch 2/15
60000/60000 [==============================] - 1s 22us/sample - loss: 0.5157 - acc: 0.9319
Epoch 3/15
60000/60000 [==============================] - 1s 22us/sample - loss: 0.3000 - acc: 0.9495
Epoch 4/15
60000/60000 [==============================] - 1s 22us/sample - loss: 0.2149 - acc: 0.9579
Epoch 5/15
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1681 - acc: 0.9654
Epoch 6/15
60000/60000 [==============================] - 1s 20us/sample - loss: 0.1510 - acc: 0.9686
Epoch 7/15
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1300 - acc: 0.9713
Epoch 8/15
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1368 - acc: 0.9709
Epoch 9/15
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1273 - acc: 0.9724
Epoch 10/15
60000/60000 [==============================] - 1s 20us/sample - loss: 

In [5]:
X = tf.placeholder(dtype=tf.float64)

Y = tf.placeholder(dtype=tf.float64)

In [7]:
num_hidden=64

# Build a hidden layer
W_hidden = tf.Variable(np.random.randn(784, num_hidden))
b_hidden = tf.Variable(np.random.randn(num_hidden))
p_hidden = tf.nn.sigmoid( tf.add(tf.matmul(X, W_hidden), b_hidden) )

W_hidden2 = tf.Variable(np.random.randn(num_hidden, num_hidden))
b_hidden2 = tf.Variable(np.random.randn(num_hidden))
p_hidden2 = tf.nn.sigmoid( tf.add(tf.matmul(p_hidden, W_hidden2), b_hidden2) )


In [8]:
# Build the output layer
W_output = tf.Variable(np.random.randn(num_hidden, 10))
b_output = tf.Variable(np.random.randn(10))
p_output = tf.nn.softmax( tf.add(tf.matmul(p_hidden2, W_output), b_output) )

In [9]:
loss = tf.reduce_mean(tf.losses.mean_squared_error(
        labels=Y,predictions=p_output))
accuracy=1-tf.sqrt(loss)

W0731 11:43:33.709659 140511422424832 deprecation.py:323] From /home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
minimization_op = tf.train.AdamOptimizer(learning_rate=0.025).minimize(loss)
saver = tf.train.Saver()


In [16]:
def norm(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

feed_dict = {
    X: norm(x_train.reshape(-1,784)[0:1000]),
    Y: pd.get_dummies(y_train[0:1000])
}

In [17]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for step in range(10000):
        J_value = session.run(loss, feed_dict)
        acc = session.run(accuracy, feed_dict)
        if step % 100 == 0:
            print("Step:", step, " Loss:", J_value," Accuracy:", acc)
            if acc > 0.88:
                save_path = saver.save(session, "/home/rubensvectomobile/TF_Scratch/model.ckpt")
                print("Model saved in path: %s" % save_path)

            session.run(minimization_op, feed_dict)
    pred00 = session.run([p_output], feed_dict={X: x_test.reshape(-1,784)})



Step: 0  Loss: 0.14142224  Accuracy: 0.6239385
Step: 100  Loss: 0.13328616  Accuracy: 0.6349162
Step: 200  Loss: 0.12372771  Accuracy: 0.64825046
Step: 300  Loss: 0.11115061  Accuracy: 0.66660744
Step: 400  Loss: 0.103042394  Accuracy: 0.6789978
Step: 500  Loss: 0.098074585  Accuracy: 0.68683136
Step: 600  Loss: 0.09329395  Accuracy: 0.69455945
Step: 700  Loss: 0.08860757  Accuracy: 0.70232975
Step: 800  Loss: 0.08391378  Accuracy: 0.71032125
Step: 900  Loss: 0.0805539  Accuracy: 0.71617985
Step: 1000  Loss: 0.080194935  Accuracy: 0.7168129
Step: 1100  Loss: 0.07456395  Accuracy: 0.726936
Step: 1200  Loss: 0.069983445  Accuracy: 0.73545617
Step: 1300  Loss: 0.066679396  Accuracy: 0.74177647
Step: 1400  Loss: 0.06306714  Accuracy: 0.7488683
Step: 1500  Loss: 0.059691772  Accuracy: 0.75568104
Step: 1600  Loss: 0.056576617  Accuracy: 0.7621416
Step: 1700  Loss: 0.053472273  Accuracy: 0.76875925
Step: 1800  Loss: 0.051082294  Accuracy: 0.7739861
Step: 1900  Loss: 0.048415538  Accuracy: 0.7

Step: 8800  Loss: 0.0013207117  Accuracy: 0.9636584
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 8900  Loss: 0.0012938918  Accuracy: 0.9640293
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9000  Loss: 0.0012739706  Accuracy: 0.96430725
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9100  Loss: 0.0012521858  Accuracy: 0.96461374
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9200  Loss: 0.0012289435  Accuracy: 0.9649437
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9300  Loss: 0.0012092011  Accuracy: 0.9652264
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9400  Loss: 0.001195855  Accuracy: 0.9654189
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9500  Loss: 0.0011877834  Accuracy: 0.96553576
Model saved in path: /home/rubensvectomobile/TF_Scratch/model.ckpt
Step: 9600  Loss: 0.0011827949  Accuracy: 0.96

In [18]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    W_hidden_value = session.run(W_hidden)
    print("Hidden layer weight matrix:\n", W_hidden_value)
    b_hidden_value = session.run(b_hidden)
    print("Hidden layer bias:\n", b_hidden_value)
    W_output_value = session.run(W_output)
    print("Output layer weight matrix:\n", W_output_value)
    b_output_value = session.run(b_output)
    print("Output layer bias:\n", b_output_value)

Hidden layer weight matrix:
 [[-1.13167696  0.74211726 -1.70574024 ...  0.32942568 -1.64003583
   0.71697142]
 [ 0.70593795  1.20575551  0.87771296 ... -1.21610123 -0.36494929
   1.09191687]
 [-0.26546801 -0.82100823  0.39032903 ...  0.80766464  0.60827513
   0.30419993]
 ...
 [ 1.77193266 -0.45261905  1.95183448 ...  0.70219883  1.62040904
   0.6158453 ]
 [ 1.15547372 -1.86500593 -0.78948615 ...  0.78806477  0.22266454
   0.5094187 ]
 [ 0.36049911  0.07560936 -0.03687464 ...  1.43193259  0.76366507
   0.904425  ]]
Hidden layer bias:
 [-0.4412009  -1.67690763 -0.80200238 -0.38237946  0.88290632  1.97719905
  0.62508441  0.68444663  0.82888819  0.91238737  2.26789438  0.08879024
 -1.50814868 -0.5837291  -0.11673317  0.26775347 -0.10983679  0.19881108
  0.02460509  2.41315083  1.18718118  1.11592968 -0.66924339 -0.59652604
  0.0667516  -1.66749556  0.58361337  0.73854025  0.22863967  0.82687444
 -0.09532057 -0.38357168  1.35476993  0.61359474 -1.76098758  0.99646026
  0.92632741  1.11215

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, np.argmax(np.array(pred00).reshape(10000,-1),axis=1))

array([[ 882,    2,   22,   17,    6,   15,   13,   15,    3,    5],
       [   0, 1066,   12,    8,    0,    3,    4,    3,   39,    0],
       [  16,   14,  828,   27,   14,    9,   44,   21,   52,    7],
       [   9,    2,   39,  742,    1,  126,    5,   16,   58,   12],
       [   0,    0,    6,   25,  717,    4,   27,    6,   15,  182],
       [  20,    2,    8,   88,   23,  616,   14,   16,   65,   40],
       [  48,    7,   38,    3,   25,   34,  779,    4,   17,    3],
       [   1,   10,   27,   23,    9,    3,    2,  865,   11,   77],
       [  26,   14,   37,   48,   18,   33,   19,   32,  711,   36],
       [  11,    1,   13,   30,   43,   13,    4,   68,   15,  811]])